# path_transforms

> Helpers for applying 2D transforms to paths.

In [ ]:
#| default_exp stroke3.path_transforms

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# |hide

In [ ]:
#| export
import numpy as np

from singleline_dataset.stroke3.transforms import apply_transform, bb_rank2


def rescale_strokes(all_strokes, target_size):
    # even though we want to default to keeping paths separate (until we can do bbox checking),
    # we do want a global max/min coord so that we can rescale all points within the same space.
    vstack_coords = np.vstack(all_strokes)
    # print(f"vstack_coords.shape={vstack_coords.shape}")
    vstack_bounding_box = bb_rank2(vstack_coords)
    vstack_rescale_transform = vstack_bounding_box.normalization_xform(target_size)

    all_rescaled_strokes = [
        apply_transform(c[:, :2], vstack_rescale_transform)[:, :2] for c in all_strokes
    ]
    return np.array(all_rescaled_strokes, dtype=object)

In [ ]:
#| export
def transform_paths(paths, global_transform=None):
    if not global_transform:
        return paths
    transformed_strokes = []
    for path_strokes in paths:
        strokes = [apply_transform(s, global_transform) for s in path_strokes]
        transformed_strokes.append(strokes)
    return transformed_strokes

In [ ]:
#| export
from singleline_dataset.stroke3.svg_files import load_svg
from singleline_dataset.stroke3.svg_transforms import svg_to_transforms
from singleline_dataset.stroke3.path_helpers import discretize_paths



def svg_to_strokes(input_fname, target_size=200, total_n=1000, min_n=3):
    paths, attributes, svg_attributes, svg_root = load_svg(input_fname)
    all_strokes = discretize_paths(paths, total_n=total_n, min_n=min_n)

    globally_rescaled_strokes = transform_paths(
        all_strokes, global_transform=svg_to_transforms(svg_root)
    )

    rescaled_strokes = rescale_strokes(globally_rescaled_strokes, target_size)
    return rescaled_strokes


In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()